# Código de R en colab

# Abriendo cuestiones de python

In [ ]:
!pip install gspread
!pip install rpy2==3.5.1
from google.colab import auth
from google.colab import drive
drive.mount('/content/drive/')
import gspread
from google.auth import default
import pandas as pd
import numpy as np
%load_ext rpy2.ipython
auth.authenticate_user()
creds, _ = default() # default pertenece a google.auth. Genera una tupla, y solo nos importa el primer elemento.
gc = gspread.authorize(creds)

In [15]:
# Parte de la base madre:

bs_madre_lm = gc.open('Links datos lectura mensual asesorados 2023').sheet1 # Acá defino la hoja que quiero abrir
bs_madre_lm = bs_madre_lm.get_all_values() #get_all_values gives a list of rows
bs_madre_lm = pd.DataFrame(bs_madre_lm,) #Convert to a DataFrame
bs_madre_lm.columns = bs_madre_lm.iloc[0]
bs_madre_lm = bs_madre_lm[1:]
bs_madre_lm = bs_madre_lm.iloc[:,0:4]
bs_madre_lm["nombre_arch"] = "Planilla registro lecturas mensuales " + bs_madre_lm.Colegio + " " + bs_madre_lm.Año
bs_madre_lm = bs_madre_lm.loc[bs_madre_lm.Año == "2023"] # Se puede modificar o agregar un filtro en la base madre


lista_compilados = []

for n in range(len(bs_madre_lm.nombre_arch.to_list())):
  try:
    archivo = bs_madre_lm.nombre_arch.to_list()[n]
    Colegio = bs_madre_lm.Colegio.to_list()[n]
    Año = bs_madre_lm.Año.to_list()[n]
    hojas = gc.open(archivo)
    num_hojas = len(hojas.worksheets()) # Obtener el número de hojas en el archivo
    nombres_hojas = [hojas.get_worksheet(i).title for i in range(num_hojas)] # Obtener los nombres de las hojas en el archivo
    nombres_hojas = [hoja for hoja in nombres_hojas if hoja in ["1A","1B","1C","1D","1E"]]
    for hoja in  nombres_hojas:
      rows = hojas.worksheet(hoja)
      rows = rows.get_all_values()[6:53]
      rows = [fila[:13] for fila in rows]
      df = pd.DataFrame(rows)
      df.columns = df.iloc[0]
      df = df[1:]
      df["Año"] = Año
      df["Colegio"] = Colegio
      df["Letra"] = hoja
      lista_compilados.append(df)
      print("planilla " + archivo + " " + hoja + " correctamente compilada")
  except:
    print("error en "+archivo)

compilado_lm = pd.concat(lista_compilados, ignore_index=True)

# Si se quisiera revisar alguna base:
#rows = gc.open(i).sheet1
#rows = rows.get_all_values()[6:53]
#rows = [fila[:13] for fila in rows]



planilla Planilla registro lecturas mensuales EBPA 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales EBPA 2023 1B correctamente compilada
planilla Planilla registro lecturas mensuales JF 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales JF 2023 1B correctamente compilada
planilla Planilla registro lecturas mensuales JF 2023 1C correctamente compilada
planilla Planilla registro lecturas mensuales MR 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales MR 2023 1B correctamente compilada
planilla Planilla registro lecturas mensuales SVF 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales SVF 2023 1B correctamente compilada
error en Planilla registro lecturas mensuales CSFP 2023
planilla Planilla registro lecturas mensuales CMA 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales ELA 2023 1A correctamente compilada
planilla Planilla registro lecturas mensuales

In [ ]:
compilado_lm # solo para ver si hay problemas en la base

In [16]:
# depurar compilado:
## Borrar los valores faltantes de los nombres:
compilado_lm["Nombre"] = compilado_lm["Nombre"].replace('', np.nan)
compilado_lm.dropna(subset="Nombre",inplace=True)
## Resetear el índice
compilado_lm = compilado_lm.reset_index(drop=True)
## Seleccionar columnas:
compilado_lm = compilado_lm.iloc[:,0:16]
## Cambiar nombres:
compilado_lm.columns = ["N","Nombre","Lectura_1","Lectura_2","Lectura_3","Lectura_4","Lectura_5","Lectura_6","Lectura_7","Lectura_8","Lectura_9","Cantidad_palabras","Categoría","Año","Colegio","Letra"]
compilado_lm = compilado_lm.iloc[:,[13,14,15,0,1,2,3,4,5,6,7,8,9,10,11,12]]


# Abriendo R

In [17]:
%R library(tidyverse)
%Rpush compilado_lm

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:54: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [18]:
%%R
base_resumen = compilado_lm %>% gather(key=Lectura, value=Categoría, 6:16) %>%
      mutate(Categoría = ifelse(is.na(Categoría)==T,"Ausente",Categoría)) %>%
      mutate(Categoría = ifelse(Categoría=="" | Categoría==" ","Ausente",Categoría)) %>%
      filter(Categoría %in% c("E","MB","B","S","I", "NL", "Ausente")) %>%
      count(Año, Colegio, Letra,Lectura,Categoría) %>%
      group_by(Año, Colegio, Letra,Lectura) %>% mutate(Porcentaje = round(n/sum(n,na.rm=T)*100,2)) %>% 
      ungroup() %>% 
      select(Año, Colegio, Letra, Lectura, Categoría, Porcentaje)

In [19]:
%R -o base_resumen # transformar esta base en un output de R que se puede leer en python

In [ ]:
# Ahora vamos a hacer bases resumen en base a los presentes
from itertools import product
## Hacer formato long
compilado_lm_long = compilado_lm.melt(id_vars = ["N","Año","Colegio","Letra","Nombre"], value_vars=["Lectura_1","Lectura_2","Lectura_3","Lectura_4",
                                                "Lectura_5","Lectura_6","Lectura_7","Lectura_8",
                                                "Lectura_9","Cantidad_palabras","Categoría"], var_name='Lectura', value_name='Categoría')
# Reemplazar valores ''
compilado_lm_long["Categoría"] = compilado_lm_long["Categoría"].replace('', "Ausente")
# Eliminar ausentes
compilado_lm_long = compilado_lm_long[np.logical_and(compilado_lm_long["Categoría"] != "Ausente", ~compilado_lm_long["Categoría"].isna())]


# Contar cada categoría
frecuencia = compilado_lm_long.groupby(['Año','Colegio', 'Letra', 'Lectura', 'Categoría'])['Categoría'].count().reset_index(name='Frecuencia')
# Generar grid:
Año = compilado_lm_long["Año"].unique()
Colegio = compilado_lm_long["Colegio"].unique()
Letra = compilado_lm_long["Letra"].unique()
Lectura = compilado_lm_long["Lectura"].unique()
Categoría = ['E', 'MB', 'B', 'S', 'I', "NL"]
df_completo = pd.DataFrame(list(product(Año, Colegio, Letra, Lectura, Categoría)), columns=["Año", "Colegio", "Letra", "Lectura", "Categoría"])
# Hacer el left join:
frecuencia_completa = pd.merge(df_completo, frecuencia, on=['Año', 'Colegio', 'Letra', "Lectura", 'Categoría'], how='left')
frecuencia_completa["Frecuencia"] = frecuencia_completa["Frecuencia"].fillna(0)
frecuencia_completa["Porcentaje_frecuencia"] = frecuencia_completa.groupby(['Año','Colegio', 'Letra', 'Lectura'])["Frecuencia"].apply(lambda x: 100 * x / float(x.sum()))
frecuencia_completa["Porcentaje_frecuencia"] = frecuencia_completa["Porcentaje_frecuencia"].round(2)

orden_categorias = ['E', 'MB', 'B', 'S', 'I', 'NL']
frecuencia_completa['Categoría'] = pd.Categorical(frecuencia_completa['Categoría'], categories=orden_categorias, ordered=True)
frecuencia_completa = frecuencia_completa.sort_values(['Año', 'Colegio', 'Letra', 'Lectura', 'Categoría'])

tabla_porcentajes = frecuencia_completa.pivot_table(index=['Año', 'Colegio', 'Letra',  'Categoría'], 
                                               columns='Lectura', 
                                               values='Porcentaje_frecuencia')
# resetear índice
tabla_porcentajes = tabla_porcentajes.reset_index()

tabla_frecuencias = frecuencia_completa.pivot_table(index=['Año', 'Colegio', 'Letra',  'Categoría'], 
                                               columns='Lectura', 
                                               values='Frecuencia')
# resetear índice
tabla_frecuencias = tabla_frecuencias.reset_index()


In [21]:
with pd.ExcelWriter("Compilados lectura mensual.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    compilado_lm.to_excel(writer, sheet_name="compilado", index=False)
    base_resumen.to_excel(writer, sheet_name="resumen general", index=False)
    tabla_porcentajes.to_excel(writer, sheet_name="porcentajes", index=False)
    tabla_frecuencias.to_excel(writer, sheet_name="frecuencias", index=False)